In [11]:
import openai
import requests
import pandas as pd
from dotenv import load_dotenv
import os

load_dotenv()
#api keys: alphavantage_api_key, openai_api_key

openai_api_key = os.getenv('OPENAI_API_KEY')
alphavantage_api_key = os.getenv('ALPHAVANTAGE_API_KEY')



In [30]:
openai.api_key = openai_api_key

# Define client profile
client_profile = {
    'risk_tolerance': 'low',
    'preferred_sectors': ['technology', 'healthcare'],
    'investment_horizon': 'long-term',
    'current_portfolio': ['AAPL', 'MSFT', 'GOOGL']
}




In [34]:
# Function to get stock data
def get_stock_data(symbol, api_key):
    url = f'https://www.alphavantage.co/query?function=TIME_SERIES_WEEKLY_ADJUSTED&symbol={symbol}&apikey={api_key}'
    response = requests.get(url)
    data = response.json()
    # print(data)
    df = pd.DataFrame(data['Weekly Adjusted Time Series']).T
    df = df.astype(float)
    return df[:10]

In [35]:
get_stock_data('AAPL', alphavantage_api_key)

,1. open,2. high,3. low,4. close,5. adjusted close,6. volume,7. dividend amount
2024-07-25,227.010,227.7800,214.620,217.49,217.49,201330870.0,0.0
2024-07-19,236.480,237.2300,222.270,224.31,224.31,278397452.0,0.0
2024-07-12,227.090,233.0800,223.250,230.54,230.54,287640514.0,0.0
2024-07-05,212.090,226.4500,211.920,226.34,226.34,216231316.0,0.0
2024-06-28,207.720,216.0700,206.590,210.62,210.62,335969485.0,0.0
2024-06-21,213.370,218.9500,207.110,207.49,207.49,506265358.0,0.0
2024-06-14,196.900,220.2000,192.150,212.49,212.49,635755143.0,0.0
2024-06-07,192.900,196.9400,192.520,196.89,196.89,245994434.0,0.0
2024-05-31,191.510,193.0000,189.100,192.25,192.25,230454285.0,0.0
2024-05-24,189.325,192.8231,186.625,189.98,189.98,208652122.0,0.0


In [42]:
# Function to generate recommendations using OpenAI
def generate_recommendations(client_profile, stock_data):
    prompt = f"""
    Based on my portfolio stock data, suggest if I should buy or sell the following stocks: {', '.join(client_profile['current_portfolio'])}
    Additionally given my profile and stock data, recommend stocks to me:

    Portfolio Profile:
    Risk Tolerance: {client_profile['risk_tolerance']}
    Preferred Sectors: {', '.join(client_profile['preferred_sectors'])}
    Investment Horizon: {client_profile['investment_horizon']}
    Current Portfolio: {', '.join(client_profile['current_portfolio'])}

    Stock Data:
    {stock_data}

    Recommendations:
    """
    
    response = openai.chat.completions.create(
    model="gpt-4",
    messages=[
        {
            "role": "user",
            "content": prompt,
        },
    ],
    )
    # print(response)
    return response


In [43]:

symbols = client_profile['current_portfolio']


# Get stock data for symbols
stock_data = {symbol: get_stock_data(symbol, alphavantage_api_key) for symbol in symbols}

# Prepare stock data in a simplified format for the prompt
simplified_stock_data = {symbol: stock_data[symbol].iloc[:5].to_dict() for symbol in symbols}
print(simplified_stock_data)

# Generate stock recommendations
saved = generate_recommendations(client_profile, simplified_stock_data)
print(saved.choices[0].message.content)

{'AAPL': {'1. open': {'2024-07-25': 227.01, '2024-07-19': 236.48, '2024-07-12': 227.09, '2024-07-05': 212.09, '2024-06-28': 207.72}, '2. high': {'2024-07-25': 227.78, '2024-07-19': 237.23, '2024-07-12': 233.08, '2024-07-05': 226.45, '2024-06-28': 216.07}, '3. low': {'2024-07-25': 214.62, '2024-07-19': 222.27, '2024-07-12': 223.25, '2024-07-05': 211.92, '2024-06-28': 206.59}, '4. close': {'2024-07-25': 217.49, '2024-07-19': 224.31, '2024-07-12': 230.54, '2024-07-05': 226.34, '2024-06-28': 210.62}, '5. adjusted close': {'2024-07-25': 217.49, '2024-07-19': 224.31, '2024-07-12': 230.54, '2024-07-05': 226.34, '2024-06-28': 210.62}, '6. volume': {'2024-07-25': 201330870.0, '2024-07-19': 278397452.0, '2024-07-12': 287640514.0, '2024-07-05': 216231316.0, '2024-06-28': 335969485.0}, '7. dividend amount': {'2024-07-25': 0.0, '2024-07-19': 0.0, '2024-07-12': 0.0, '2024-07-05': 0.0, '2024-06-28': 0.0}}, 'MSFT': {'1. open': {'2024-07-25': 441.79, '2024-07-19': 453.3, '2024-07-12': 466.55, '2024-07-

In [49]:
saved.choices[0].message.content

"Based on the given portfolio stock data and your profile, following is the recommendation:\n\nAAPL: Looking at the trend over the previous month, although there seems to be a slight dip in the most recent week, there has been a general upward price movement. Given your long-term investment horizon and low risk tolerance, it is recommended to HOLD the AAPL stock and monitor closely.\n\nMSFT: The stock seems to be showing a slight downward trend recently which could be a cause for concern. However, it is still in your preferred sector and given it's strong performance otherwise and your long-term investment horizon, it could be beneficial to HOLD MSFT.\n\nGOOGL: This stock seems volatile with price fluctuating up and down over the weeks. While it has seen a sharp drop in the most recent week, it has shown resilience previously. Since Google is a strong player in the technology sector, and your profile matches with this. Hence, you should HOLD to the stock.\n\nIt is worth noting that wit

In [5]:
import pandas as pd

df = pd.read_excel("adjusted.xlsx")

In [6]:
df.head()

,Date,9994967D US Equity,A US Equity,AA US Equity,AAON US Equity,AAP US Equity,AAPL US Equity,AAT US Equity,AAWW US Equity,ABCB US Equity,...,YORW US Equity,YUM US Equity,ZBH US Equity,ZBRA US Equity,ZD US Equity,ZEUS US Equity,ZG US Equity,ZION US Equity,ZUMZ US Equity,ZWS US Equity
0,2013-02-01 00:00:00,9.8461,27.2751,18.9669,5.8282,66.095,16.791,20.4463,45.58,11.4401,...,14.4082,39.3209,60.1612,40.9600,24.2131,22.0583,8.8251,17.8013,19.5000,10.6410
1,2013-03-01 00:00:00,10.0616,27.3728,19.1356,5.9295,66.095,16.579,20.0665,44.76,11.4044,...,14.2405,39.5668,61.0232,41.0000,24.1369,21.3804,8.9249,17.8256,21.2200,10.4255
2,2013-04-01 00:00:00,10.0022,27.9134,19.5365,5.9597,67.125,16.117,20.2457,45.19,11.5025,...,14.3922,40.0001,61.3342,40.6716,24.2816,21.1827,9.2305,18.3356,21.4800,10.3765
3,2013-07-01 00:00:00,9.5712,27.7115,19.1989,5.8939,66.897,16.023,20.2958,44.84,11.4768,...,14.2565,39.7483,61.4853,40.9000,24.1826,21.1168,9.8542,18.2465,20.9794,10.0090
4,2013-08-01 00:00:00,9.7495,27.4900,19.1989,5.9076,65.803,16.066,20.3532,45.53,11.5025,...,14.2964,38.0797,61.5741,40.9300,24.2511,20.4295,10.0164,17.8822,21.1200,9.7788


In [7]:
df['Date'] = pd.to_datetime(df['Date'])
df_first_of_month = df.groupby(df['Date'].dt.to_period('M')).first()

In [15]:
df_first_of_month.head()

,9994967D US Equity,A US Equity,AA US Equity,AAON US Equity,AAP US Equity,AAPL US Equity,AAT US Equity,AAWW US Equity,ABCB US Equity,ABG US Equity,...,YORW US Equity,YUM US Equity,ZBH US Equity,ZBRA US Equity,ZD US Equity,ZEUS US Equity,ZG US Equity,ZION US Equity,ZUMZ US Equity,ZWS US Equity
Date,,,,,,,,,,,,,,,,,,,,,
2013-01,9.6009,28.3562,18.8192,6.1981,66.177,15.345,20.3532,44.79,11.4133,32.39,...,14.5919,38.9301,63.1382,40.9800,24.5101,20.5331,10.3033,17.6961,20.19,9.4995
2013-02,9.8461,27.2751,18.9669,5.8282,66.095,16.791,20.4463,45.58,11.4401,32.26,...,14.4082,39.3209,60.1612,40.9600,24.2131,22.0583,8.8251,17.8013,19.50,10.6410
2013-03,10.0616,27.3728,19.1356,5.9295,66.095,16.579,20.0665,44.76,11.4044,33.03,...,14.2405,39.5668,61.0232,41.0000,24.1369,21.3804,8.9249,17.8256,21.22,10.4255
2013-04,10.0022,27.9134,19.5365,5.9597,67.125,16.117,20.2457,45.19,11.5025,34.41,...,14.3922,40.0001,61.3342,40.6716,24.2816,21.1827,9.2305,18.3356,21.48,10.3765
2013-05,10.5595,29.3071,18.8614,6.3789,66.806,14.002,20.9838,44.52,11.9662,37.11,...,15.0711,36.5349,65.8130,43.4200,23.7713,19.4222,11.8282,19.4608,21.13,9.7690


In [9]:
# df_first_of_month.to_csv('smaller_adjusted.csv', index=False)

In [27]:
# df_first_of_month.drop('Date', axis=1, inplace=True)
df_pct_change = df_first_of_month.pct_change()*100
df_pct_change = df_pct_change.iloc[1:]

C:\Users\Raph\AppData\Local\Temp\ipykernel_13108\103151936.py:2: FutureWarning: The default fill_method='pad' in DataFrame.pct_change is deprecated and will be removed in a future version. Call ffill before calling pct_change to retain current behavior and silence this warning.
  df_pct_change = df_first_of_month.pct_change()*100


In [28]:
# df_pct_change.to_csv('smaller_adjusted_pct_change.csv', index=True)
df_pct_change.head()

,9994967D US Equity,A US Equity,AA US Equity,AAON US Equity,AAP US Equity,AAPL US Equity,AAT US Equity,AAWW US Equity,ABCB US Equity,ABG US Equity,...,YORW US Equity,YUM US Equity,ZBH US Equity,ZBRA US Equity,ZD US Equity,ZEUS US Equity,ZG US Equity,ZION US Equity,ZUMZ US Equity,ZWS US Equity
Date,,,,,,,,,,,,,,,,,,,,,
2013-02,2.553927,-3.812570,0.784837,-5.967958,-0.123910,9.423265,0.457422,1.763787,0.234814,-0.401358,...,-1.258918,1.003850,-4.715054,-0.048804,-1.211745,7.428006,-14.346860,0.594481,-3.417533,12.016422
2013-03,2.188684,0.358202,0.889444,1.738101,0.000000,-1.262581,-1.857549,-1.799035,-0.312060,2.386857,...,-1.163921,0.625367,1.432817,0.097656,-0.314706,-3.073220,1.130865,0.136507,8.820513,-2.025186
2013-04,-0.590363,1.974953,2.095048,0.509318,1.558363,-2.786658,0.893031,0.960679,0.860194,4.178020,...,1.065272,1.095110,0.509642,-0.800976,0.599497,-0.924679,3.424128,2.861054,1.225259,-0.470001
2013-05,5.571774,4.992942,-3.455583,7.033911,-0.475233,-13.122790,3.645712,-1.482629,4.031298,7.846556,...,4.717138,-8.662978,7.302288,6.757541,-2.101591,-8.311027,28.142571,6.136696,-1.629423,-5.854575
2013-06,1.969790,0.199952,0.560934,1.932935,-0.504446,-0.107128,-0.136296,-0.494160,0.297505,-1.778496,...,0.317827,1.369102,1.417653,1.174574,0.224220,3.732327,-3.823067,0.000000,4.827260,1.955164


In [30]:
import numpy as np

In [38]:
returns = df_first_of_month.pct_change()
daily_volatility = returns.std()


C:\Users\Raph\AppData\Local\Temp\ipykernel_13108\3175528684.py:1: FutureWarning: The default fill_method='pad' in DataFrame.pct_change is deprecated and will be removed in a future version. Call ffill before calling pct_change to retain current behavior and silence this warning.
  returns = df_first_of_month.pct_change()


In [39]:
daily_volatility.head()

 9994967D US Equity    0.152230
A US Equity            0.080260
AA US Equity           0.206467
AAON US Equity         0.121969
AAP US Equity          0.091492
dtype: float64

In [43]:
# Define the rolling window size
window_size = 3

# Calculate rolling volatility (standard deviation of daily returns)
rolling_volatility = returns.rolling(window=window_size).std()

In [44]:
rolling_volatility= rolling_volatility.iloc[window_size:]
rolling_volatility.head()

,9994967D US Equity,A US Equity,AA US Equity,AAON US Equity,AAP US Equity,AAPL US Equity,AAT US Equity,AAWW US Equity,ABCB US Equity,ABG US Equity,...,YORW US Equity,YUM US Equity,ZBH US Equity,ZBRA US Equity,ZD US Equity,ZEUS US Equity,ZG US Equity,ZION US Equity,ZUMZ US Equity,ZWS US Equity
Date,,,,,,,,,,,,,,,,,,,,,
2013-04,0.017196,0.029862,0.007281,0.041402,0.009375,0.066532,0.014784,0.018688,0.005866,0.023077,...,0.013153,0.002491,0.033153,0.004821,0.009056,0.055477,0.096663,0.014589,0.061781,0.076974
2013-05,0.030860,0.023524,0.029195,0.034671,0.010638,0.064527,0.027516,0.015103,0.022470,0.027831,...,0.029691,0.055032,0.036843,0.041290,0.013738,0.037993,0.149772,0.030043,0.054012,0.027712
2013-06,0.030957,0.024232,0.028663,0.034307,0.011826,0.068729,0.019553,0.012290,0.020130,0.048576,...,0.023540,0.057146,0.036877,0.039201,0.014632,0.060730,0.167597,0.030707,0.032355,0.039972
2013-07,0.087783,0.053215,0.025313,0.083870,0.006547,0.138478,0.034035,0.013669,0.042115,0.079764,...,0.052320,0.080806,0.076557,0.068651,0.018273,0.072853,0.217447,0.061883,0.051203,0.041515
2013-08,0.075211,0.031195,0.006115,0.060861,0.011396,0.083597,0.018708,0.010932,0.026763,0.049452,...,0.034662,0.057631,0.050418,0.043772,0.007217,0.043795,0.076036,0.031866,0.050826,0.021618


In [45]:
rolling_volatility.to_csv('rolling_volatility.csv')